<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/AKSHARE01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 用 Python 快速获取基金的股票持仓增减情况

AKShare 是基于 Python 的财经数据接口库，目的是实现对股票、期货、期权、基金、外汇、债券、指数、加密货币等金融产品的基本面数据、实时和历史行情数据、衍生数据从数据采集、数据清洗到数据落地的一套工具。


本文中，给大家分享的主题是 用 Python 来追踪和更新基金的持仓结构以及基金的股票增持和减持情况，涉及到的Python库主要是 pandas 和 akshare 。

最终实现的效果包括两个方面：

*   单支基金的不同季度持仓变化情况
*   多支基金的十大持仓的历史信息



In [1]:
!pip install akshare

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import akshare as ak
import pandas as pd
import numpy as np

print(f'pandas version: {pd.__version__}')
print(f'akshare version: {ak.__version__}')

pandas version: 1.3.5
akshare version: 1.7.47


### 单支基金
对于单支基金的持仓情况变化，主要是通过对比临近的两个季度，来观察基金持仓的股票增持与减持情况，从而来跟踪基金的趋势变化。

由于基金季度报告中，1季度和3季度只披露前十大股票的持仓情况，而半年度（Q2）和年度（Q4）报告会披露全部持仓的情况，因此，也可以通过对更详细信息的对比来挖掘有些趋势信息。

下面，将详细的介绍获取并对比两个季度的持仓数据。

#### 分步骤介绍

首先，咱们通过 akshare 来获取基金的季度持仓数据，以 易方达蓝筹精选基金（基金代码：005827）为例，获取2019-2021年的持仓数据情况，代码如下：

In [3]:
# 易方达蓝筹精选，005827
# 前海开源公共事业股票，005669
years = ['2019','2020','2021']

data = pd.DataFrame()
for yr in years:
    # df_tmp = ak.fund_em_portfolio_hold(code="005827",year=yr) # akshare version: 1.0.91
    # df_tmp = ak.fund_portfolio_hold_em(code="005827",year=yr) # akshare version: 1.3.41
    df_tmp = ak.fund_portfolio_hold_em(symbol="005827",date=yr) # akshare version: 1.7.46
    data = data.append(df_tmp)

data['季度'] = data['季度'].apply(lambda x:x[:6])
data['季度'] = data['季度'].str.replace('年','Q')
data['占净值比例'] = pd.to_numeric(data['占净值比例'])
data

,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,1,00700,腾讯控股,10.05,251.50,84618.42,2019Q4
1,2,000568,泸州老窖,9.96,968.00,83906.24,2019Q4
2,3,01177,中国生物制药,9.79,8448.10,82487.27,2019Q4
3,4,000858,五粮液,9.77,619.01,82333.96,2019Q4
4,5,600519,贵州茅台,9.73,69.26,81938.84,2019Q4
...,...,...,...,...,...,...,...
200,201,002007,华兰生物,2.80,6200.01,246450.43,2021Q1
201,202,002044,美年健康,1.87,10660.01,164377.28,2021Q1
202,203,600872,中炬高新,1.71,3100.01,150133.56,2021Q1
203,204,300015,爱尔眼科,1.35,2000.00,118500.23,2021Q1


假设咱们需要对比2021年1季度和2021年2季度的持仓变化情况，则只需要在上面的数据上选择 2021Q1 和 2021Q2 即可。

数据选取如下：



In [4]:
s1 = '2021Q1'
s2 = '2021Q2'
s1_share = s1+'持股数'
s2_share = s2+'持股数'
s1_value = s1+'持仓市值'
s2_value = s2+'持仓市值'
s1_ratio = s1+'持仓比例'
s2_ratio = s2+'持仓比例'

df1 = data[data['季度']==s1]
df1 = df1[['股票代码','股票名称','持股数','持仓市值','占净值比例']]
df1 = df1.rename(columns={'持股数':s1_share,'持仓市值':s1_value,'占净值比例':s1_ratio})

df2 = data[data['季度']==s2]
df2 = df2[['股票代码','股票名称','持股数','持仓市值','占净值比例']]
df2 = df2.rename(columns={'持股数':s2_share,'持仓市值':s2_value,'占净值比例':s2_ratio})

print(df1)
print(df2)

       股票代码   股票名称  2021Q1持股数  2021Q1持仓市值  2021Q1持仓比例
189  000858    五粮液    3300.00   884334.00       10.05
190   00388  香港交易所    2286.00   883734.07       10.04
191  600519   贵州茅台     432.00   867888.00        9.86
192   03690   美团-W    3370.43   849458.49        9.65
193   00700   腾讯控股    1610.00   830051.28        9.43
194  000568   泸州老窖    2980.00   670560.59        7.62
195  600036   招商银行   11300.00   577430.00        6.56
196  002415   海康威视    9000.02   503100.89        5.72
197  002304   洋河股份    2500.02   411753.21        4.68
198  000001   平安银行   13000.00   286130.00        3.25
199   03968   招商银行    5000.00   250807.17        2.85
200  002007   华兰生物    6200.01   246450.43        2.80
201  002044   美年健康   10660.01   164377.28        1.87
202  600872   中炬高新    3100.01   150133.56        1.71
203  300015   爱尔眼科    2000.00   118500.23        1.35
204  600161   天坛生物    1600.01    52256.47        0.59
       股票代码   股票名称  2021Q2持股数  2021Q2持仓市值  2021Q2持仓比例
96    00388  香港交易所    2310.0

在获取了两个季度的数据后，将数据按 "股票代码" 进行拼接，在得到的 dataframe 中，可以对持股的增减情况进行判断。

需要注意的是，由于 Q2 和 Q4，即半年度和年度报告，是需要披露全部持仓的，因此在拼接后的 dataframe 中，对应列的 NaN 数据应为 0，这一点可以细细品味下。

In [5]:
df_merge = pd.merge(df1,df2,on='股票代码',how='outer')

print(df_merge)
# Q2 和 Q4，即半年度和年度报告，是需要披露全部持仓的
# 合并后，在 dataframe 中 NaN 的数据应为 0 
if s1.endswith('Q2') or s1.endswith('Q4'):
    df_merge[s1_share] = df_merge[s1_share].fillna(0)
    df_merge[s1_value] = df_merge[s1_value].fillna(0)
    df_merge[s1_ratio] = df_merge[s1_ratio].fillna(0)

if s2.endswith('Q2') or s2.endswith('Q4'):
    df_merge[s2_share] = df_merge[s2_share].fillna(0)
    df_merge[s2_value] = df_merge[s2_value].fillna(0)
    df_merge[s2_ratio] = df_merge[s2_ratio].fillna(0)


df_merge['持股数变化'] = df_merge[s2_share] - df_merge[s1_share]
df_merge = df_merge.sort_values(s2_value,ascending=False)


df_merge['股票名称'] = df_merge['股票名称_y']

df_merge.loc[df_merge['股票名称'].isna(),'股票名称'] = df_merge.loc[df_merge['股票名称'].isna(),'股票名称_x']
df_merge = df_merge[['股票代码', '股票名称', 
                     s1_share, s1_value, s1_ratio, 
                     s2_share, s2_value, s2_ratio, 
                     '持股数变化']]

df_merge

      股票代码 股票名称_x  2021Q1持股数  2021Q1持仓市值  2021Q1持仓比例 股票名称_y  2021Q2持股数  \
0   000858    五粮液    3300.00   884334.00       10.05    五粮液    2980.00   
1    00388  香港交易所    2286.00   883734.07       10.04  香港交易所    2310.00   
2   600519   贵州茅台     432.00   867888.00        9.86   贵州茅台     426.00   
3    03690   美团-W    3370.43   849458.49        9.65   美团-W    2800.00   
4    00700   腾讯控股    1610.00   830051.28        9.43   腾讯控股    1810.00   
..     ...    ...        ...         ...         ...    ...        ...   
91  300995    NaN        NaN         NaN         NaN   奇德新材       0.02   
92  301007    NaN        NaN         NaN         NaN    德迈仕       0.03   
93  301012    NaN        NaN         NaN         NaN   扬电科技       0.02   
94  300992    NaN        NaN         NaN         NaN   泰福泵业       0.02   
95  301010    NaN        NaN         NaN         NaN   晶雪节能       0.02   

    2021Q2持仓市值  2021Q2持仓比例  
0    887712.20        9.88  
1    889550.10        9.90  
2    876154.20        9.

,股票代码,股票名称,2021Q1持股数,2021Q1持仓市值,2021Q1持仓比例,2021Q2持股数,2021Q2持仓市值,2021Q2持仓比例,持股数变化
1,00388,香港交易所,2286.00,883734.07,10.04,2310.00,889550.10,9.90,24.00
0,000858,五粮液,3300.00,884334.00,10.05,2980.00,887712.20,9.88,-320.00
4,00700,腾讯控股,1610.00,830051.28,9.43,1810.00,879541.84,9.78,200.00
2,600519,贵州茅台,432.00,867888.00,9.86,426.00,876154.20,9.75,-6.00
5,000568,泸州老窖,2980.00,670560.59,7.62,3700.01,872980.71,9.71,720.01
...,...,...,...,...,...,...,...,...,...
93,301012,扬电科技,NaN,NaN,NaN,0.02,0.41,0.00,NaN
95,301010,晶雪节能,NaN,NaN,NaN,0.02,0.38,0.00,NaN
12,002044,美年健康,10660.01,164377.28,1.87,0.00,0.00,0.00,-10660.01
11,002007,华兰生物,6200.01,246450.43,2.80,0.00,0.00,0.00,-6200.01


上面的结果中，如果没有对 Q2 和 Q4 季度进行判断，则上图中红框内的数据会是 NaN，同时也计算不出减持的结果（结果会显示 NaN）。

因此，在进行调整后，可以真实的看到基金减持的情况。在2021年2季度，易方达蓝筹精选基金清仓了中炬高新等股票。

针对上述表格，咱们也可以通过 pandas 的样式设置，来较为清晰的查看基金的增减持情况，代码如下：

In [6]:
format_dict = {s1_share: '{0:.2f}', s2_share: '{0:.2f}', 
               s1_value: '{0:.0f}', s2_value: '{0:.0f}',
               s1_ratio: '{0:.2f}', s2_ratio: '{0:.2f}', 
               '持股数变化': '{0:.2f}'}

df_merge.style.hide_index()\
              .hide_columns(['股票代码'])\
              .format(format_dict)\
              .bar(subset=['持股数变化'],color=['#99ff66','#ee7621'],align='mid')

股票名称,2021Q1持股数,2021Q1持仓市值,2021Q1持仓比例,2021Q2持股数,2021Q2持仓市值,2021Q2持仓比例,持股数变化
香港交易所,2286.00,883734,10.04,2310.00,889550,9.90,24.00
五粮液,3300.00,884334,10.05,2980.00,887712,9.88,-320.00
腾讯控股,1610.00,830051,9.43,1810.00,879542,9.78,200.00
贵州茅台,432.00,867888,9.86,426.00,876154,9.75,-6.00
泸州老窖,2980.00,670561,7.62,3700.01,872981,9.71,720.01
洋河股份,2500.02,411753,4.68,4200.01,870241,9.68,1699.99
美团-W,3370.43,849458,9.65,2800.00,746476,8.30,-570.43
招商银行,11300.00,577430,6.56,11300.00,612347,6.81,0.00
海康威视,9000.02,503101,5.72,9000.02,580501,6.46,0.00
招商银行,5000.00,250807,2.85,5000.00,275626,3.07,0.00


在上面的表格中，nan 数据结果，是由于数据披露的情况，有些数据咱们是获取不到的，因此也计算不了持仓增减情况，这些数据，在实际判断中，意义是不大的。

因此，咱们在显示数据结果的时候，也可以选择隐藏这些数据，如下：

In [7]:
df_merge_update = df_merge[~(df_merge['持股数变化'].isna())]
df_merge_update.style.hide_index()\
               .hide_columns(['股票代码'])\
               .format(format_dict)\
               .bar(subset=['持股数变化'],color=['#99ff66','#ee7621'],align='mid')

股票名称,2021Q1持股数,2021Q1持仓市值,2021Q1持仓比例,2021Q2持股数,2021Q2持仓市值,2021Q2持仓比例,持股数变化
香港交易所,2286.00,883734,10.04,2310.00,889550,9.90,24.00
五粮液,3300.00,884334,10.05,2980.00,887712,9.88,-320.00
腾讯控股,1610.00,830051,9.43,1810.00,879542,9.78,200.00
贵州茅台,432.00,867888,9.86,426.00,876154,9.75,-6.00
泸州老窖,2980.00,670561,7.62,3700.01,872981,9.71,720.01
洋河股份,2500.02,411753,4.68,4200.01,870241,9.68,1699.99
美团-W,3370.43,849458,9.65,2800.00,746476,8.30,-570.43
招商银行,11300.00,577430,6.56,11300.00,612347,6.81,0.00
海康威视,9000.02,503101,5.72,9000.02,580501,6.46,0.00
招商银行,5000.00,250807,2.85,5000.00,275626,3.07,0.00


这样，咱们就可以得到，从两个季度信息对比，可以确定的增减持股票情况了（注意，不是全部的增减持，因为有些季度，有些持仓数据是不披露的）。

#### 函数封装

将上面的详细过程封装成一个自定义函数，这样咱们就可以在后续重复使用该函数来获取不同基金的持仓变化信息了。

函数封装如下：



In [8]:
def compare(code,years,s1,s2):
    """
    code：str 基金代码
    years：list 年份列表 ['yr1','yr2','……']
    s1：str 靠前的季度 格式为 'YYYYQ1' 例如：'2021Q2'
    s2：str 靠后的季度 格式为 'YYYYQ1' 例如：'2021Q2'
    注意：s1和s2的年份应在 years 里
    """
    
    s1_share = s1+'持股数'
    s2_share = s2+'持股数'
    s1_value = s1+'持仓市值'
    s2_value = s2+'持仓市值'
    s1_ratio = s1+'持仓比例'
    s2_ratio = s2+'持仓比例'

    data = pd.DataFrame()
    for yr in years:
        # df_tmp = ak.fund_em_portfolio_hold(code="005827",year=yr) # akshare version: 1.0.91
        # df_tmp = ak.fund_portfolio_hold_em(code="005827",year=yr) # akshare version: 1.3.41
        df_tmp = ak.fund_portfolio_hold_em(symbol="005827",date=yr) # akshare version: 1.7.46
        data = data.append(df_tmp)

    data['季度'] = data['季度'].apply(lambda x:x[:6])
    data['季度'] = data['季度'].str.replace('年','Q')
    data['占净值比例'] = pd.to_numeric(data['占净值比例'])

    df1 = data[data['季度']==s1]
    df1 = df1[['股票代码','股票名称','持股数','持仓市值','占净值比例']]
    df1 = df1.rename(columns={'持股数':s1_share,'持仓市值':s1_value,'占净值比例':s1_ratio})

    df2 = data[data['季度']==s2]
    df2 = df2[['股票代码','股票名称','持股数','持仓市值','占净值比例']]
    df2 = df2.rename(columns={'持股数':s2_share,'持仓市值':s2_value,'占净值比例':s2_ratio})

    df_merge = pd.merge(df1,df2,on='股票代码',how='outer')

    # Q2 和 Q4，即半年度和年度报告，是需要披露全部持仓的
    # 合并后，在 dataframe 中 NaN 的数据应为 0 

    if s1.endswith('Q2') or s1.endswith('Q4'):
        df_merge[s1_share] = df_merge[s1_share].fillna(0)
        df_merge[s1_value] = df_merge[s1_value].fillna(0)
        df_merge[s1_ratio] = df_merge[s1_ratio].fillna(0)

    if s2.endswith('Q2') or s2.endswith('Q4'):
        df_merge[s2_share] = df_merge[s2_share].fillna(0)
        df_merge[s2_value] = df_merge[s2_value].fillna(0)
        df_merge[s2_ratio] = df_merge[s2_ratio].fillna(0)


    df_merge['持股数变化'] = df_merge[s2_share] - df_merge[s1_share]
    df_merge = df_merge.sort_values(s2_value,ascending=False)


    df_merge['股票名称'] = df_merge['股票名称_y']
    df_merge.loc[df_merge['股票名称'].isna(),'股票名称'] = df_merge.loc[df_merge['股票名称'].isna(),'股票名称_x']
    df_merge = df_merge[['股票代码', '股票名称', 
                        s1_share, s1_value, s1_ratio, 
                        s2_share, s2_value, s2_ratio, 
                        '持股数变化']]
    return df_merge

In [9]:
code = "005827"
years = ['2020','2021']
s1 = '2020Q3' 
s2 = '2021Q1'
s1_share = s1+'持股数'
s2_share = s2+'持股数'
s1_value = s1+'持仓市值'
s2_value = s2+'持仓市值'
s1_ratio = s1+'持仓比例'
s2_ratio = s2+'持仓比例'

df_merge = compare(code,years,s1,s2)

format_dict = {s1_share: '{0:.2f}', s2_share: '{0:.2f}', 
               s1_value: '{0:.0f}', s2_value: '{0:.0f}',
               s1_ratio: '{0:.2f}', s2_ratio: '{0:.2f}', 
               '持股数变化': '{0:.2f}'}

df_merge.style.hide_index()\
              .hide_columns(['股票代码'])\
              .format(format_dict)\
              .bar(subset=['持股数变化'],color=['#99ff66','#ee7621'],align='mid')

股票名称,2020Q3持股数,2020Q3持仓市值,2020Q3持仓比例,2021Q1持股数,2021Q1持仓市值,2021Q1持仓比例,持股数变化
五粮液,1454.71,321491,9.47,3300.00,884334,10.05,1845.29
香港交易所,1012.26,321819,9.48,2286.00,883734,10.04,1273.74
贵州茅台,193.56,322951,9.52,432.00,867888,9.86,238.44
美团-W,850.00,180603,5.32,3370.43,849458,9.65,2520.43
腾讯控股,718.40,322896,9.51,1610.00,830051,9.43,891.60
泸州老窖,2307.85,331292,9.76,2980.00,670561,7.62,672.15
招商银行,nan,nan,nan,11300.00,577430,6.56,nan
海康威视,3800.01,144818,4.27,9000.02,503101,5.72,5200.01
洋河股份,2585.00,323099,9.52,2500.02,411753,4.68,-84.98
平安银行,nan,nan,nan,13000.00,286130,3.25,nan


### 多支基金

同时获取多支基金的十大股票持仓信息时，在这里只获取每个季度股票名称的信息。

首先，来获取单支基金的十大持仓股票名称信息，自定义函数如下：



In [10]:
# 获取单只基金的十大股票名称信息
def fund_stock_holding(years,code):
    data = pd.DataFrame()
    for yr in years:
        # df_tmp = ak.fund_em_portfolio_hold(code="005827",year=yr) # akshare version: 1.0.91
        # df_tmp = ak.fund_portfolio_hold_em(code="005827",year=yr) # akshare version: 1.3.41
        df_tmp = ak.fund_portfolio_hold_em(symbol="005827",date=yr) # akshare version: 1.7.46
        data = data.append(df_tmp)
    
    data['季度'] = data['季度'].apply(lambda x:x[:6])
    data['季度'] = data['季度'].str.replace('年','Q')
    data['占净值比例'] = pd.to_numeric(data['占净值比例'])    
    data = data.sort_values(['季度','持仓市值'],ascending=[True,False])
    
    # 将序号按持仓市值排序，从大到小    
    data_ = pd.DataFrame()
    quarter = data['季度'].unique().tolist()
    for q in quarter:
        df_tmp = data.query('季度==@q')
        df_tmp['序号'] = list(range(1,len(df_tmp)+1)) 
        data_ = data_.append(df_tmp)
        
    df = data_.set_index(['序号','季度']).stack().unstack([1,2]).head(10)

    df = df.loc[:,(slice(None), '股票名称')]  # 只选取股票名称
    df = df.droplevel(None,axis=1)
    df.columns.name=None
    df = df.reset_index()
    df['基金代码'] = code
    cols = df.columns.tolist()
    cols = cols[:1] + cols[-1:] + cols[1:-1]  # 将基金代码列名放前面
    df = df[cols]
    return df

以 易方达蓝筹精选基金（基金代码：005827）为例，该基金2019-2021的十大持仓信息如下：

In [11]:
years = ['2020','2021']
df = fund_stock_holding(years,'005827')
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,序号,基金代码,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4
0,1,005827,腾讯控股,香港交易所,泸州老窖,贵州茅台,五粮液,香港交易所,泸州老窖,腾讯控股
1,2,005827,贵州茅台,腾讯控股,洋河股份,美团-W,香港交易所,五粮液,贵州茅台,海康威视
2,3,005827,泸州老窖,五粮液,贵州茅台,洋河股份,贵州茅台,腾讯控股,五粮液,泸州老窖
3,4,005827,五粮液,贵州茅台,腾讯控股,腾讯控股,美团-W,贵州茅台,海康威视,贵州茅台
4,5,005827,中国生物制药,泸州老窖,香港交易所,五粮液,腾讯控股,泸州老窖,香港交易所,五粮液
5,6,005827,香港交易所,美团-W,五粮液,香港交易所,泸州老窖,洋河股份,洋河股份,洋河股份
6,7,005827,华兰生物,中国生物制药,美团-W,泸州老窖,招商银行,美团-W,腾讯控股,香港交易所
7,8,005827,药明生物,药明生物,爱尔眼科,海康威视,海康威视,招商银行,招商银行,招商银行
8,9,005827,石药集团,天坛生物,中国生物制药,爱尔眼科,洋河股份,海康威视,伊利股份,伊利股份
9,10,005827,上海机场,爱尔眼科,海康威视,颐海国际,平安银行,招商银行,平安银行,平安银行


通过自定义函数 fund_stock_holding(years,code) 可以同时获取多支基金的十大持仓信息，代码如下：

In [12]:
codes = ['005669','006281','003956','001532',
         '001714','001054','001809','003853',
         '004890','001125','001808']
df = pd.DataFrame()
for code in codes:
    df_tmp = fund_stock_holding(years,code)
    df = df.append(df_tmp)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,序号,基金代码,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4
0,1,005669,腾讯控股,香港交易所,泸州老窖,贵州茅台,五粮液,香港交易所,泸州老窖,腾讯控股
1,2,005669,贵州茅台,腾讯控股,洋河股份,美团-W,香港交易所,五粮液,贵州茅台,海康威视
2,3,005669,泸州老窖,五粮液,贵州茅台,洋河股份,贵州茅台,腾讯控股,五粮液,泸州老窖
3,4,005669,五粮液,贵州茅台,腾讯控股,腾讯控股,美团-W,贵州茅台,海康威视,贵州茅台
4,5,005669,中国生物制药,泸州老窖,香港交易所,五粮液,腾讯控股,泸州老窖,香港交易所,五粮液
...,...,...,...,...,...,...,...,...,...,...
5,6,001808,香港交易所,美团-W,五粮液,香港交易所,泸州老窖,洋河股份,洋河股份,洋河股份
6,7,001808,华兰生物,中国生物制药,美团-W,泸州老窖,招商银行,美团-W,腾讯控股,香港交易所
7,8,001808,药明生物,药明生物,爱尔眼科,海康威视,海康威视,招商银行,招商银行,招商银行
8,9,001808,石药集团,天坛生物,中国生物制药,爱尔眼科,洋河股份,海康威视,伊利股份,伊利股份


上面数据中，显示的是基金代码，一般情况下，咱们还是习惯看基金名称，因此可以把基金代码换成基金名称。

基金名称信息获取如下：

In [13]:
# 获取基金名称信息
df_fund_info = ak.fund_name_em()
df_fund_info = df_fund_info.query('基金代码 in @codes')
df_fund_info = df_fund_info[['基金代码','基金简称']]
df_fund_info = df_fund_info.reset_index(drop=True)
df_fund_info

,基金代码,基金简称
0,001054,工银新金融股票A
1,001125,博时互联网主题灵活配置混合
2,001532,华安文体健康混合A
3,001714,工银文体产业股票A
4,001808,银华互联网主题灵活配置混合A
5,001809,中信建投智信物联网A
6,003853,金鹰信息产业股票A
7,003956,南方产业智选股票
8,004890,中邮健康文娱混合
9,005669,前海开源公用事业股票


有了基金名称信息后，可以用 pandas 通过拼接的方式，将基金名称信息显示出来，代码如下：

In [14]:
df_fund = pd.merge(df,df_fund_info,on='基金代码')
cols = df_fund.columns.tolist()
cols = cols[:1] + cols[-1:] + cols[2:-1]  # 将基金简称放前面
df_fund = df_fund[cols]
df_fund

,序号,基金简称,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4
0,1,前海开源公用事业股票,腾讯控股,香港交易所,泸州老窖,贵州茅台,五粮液,香港交易所,泸州老窖,腾讯控股
1,2,前海开源公用事业股票,贵州茅台,腾讯控股,洋河股份,美团-W,香港交易所,五粮液,贵州茅台,海康威视
2,3,前海开源公用事业股票,泸州老窖,五粮液,贵州茅台,洋河股份,贵州茅台,腾讯控股,五粮液,泸州老窖
3,4,前海开源公用事业股票,五粮液,贵州茅台,腾讯控股,腾讯控股,美团-W,贵州茅台,海康威视,贵州茅台
4,5,前海开源公用事业股票,中国生物制药,泸州老窖,香港交易所,五粮液,腾讯控股,泸州老窖,香港交易所,五粮液
...,...,...,...,...,...,...,...,...,...,...
105,6,银华互联网主题灵活配置混合A,香港交易所,美团-W,五粮液,香港交易所,泸州老窖,洋河股份,洋河股份,洋河股份
106,7,银华互联网主题灵活配置混合A,华兰生物,中国生物制药,美团-W,泸州老窖,招商银行,美团-W,腾讯控股,香港交易所
107,8,银华互联网主题灵活配置混合A,药明生物,药明生物,爱尔眼科,海康威视,海康威视,招商银行,招商银行,招商银行
108,9,银华互联网主题灵活配置混合A,石药集团,天坛生物,中国生物制药,爱尔眼科,洋河股份,海康威视,伊利股份,伊利股份
